In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from chroma_store import chroma_store



from langchain_community.document_loaders import PyPDFLoader

# file_path = "./code-de-travail.pdf" # https://www.legifrance.gouv.fr/download/pdf/legiOrKali?id=LEGITEXT000006072050.pdf&size=7%20Mo&pathToFile=/LEGI/TEXT/00/00/06/07/20/50/LEGITEXT000006072050/LEGITEXT000006072050.pdf&title=Code%20du%20travail
# loader = PyPDFLoader(file_path)
# data = loader.load()
# print("Document loaded")
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
# all_splits = text_splitter.split_documents(data)
# chroma_store = chroma_store.from_documents(documents=all_splits, embedding=embeddings_model)
# print("Vectors stored")


In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


from langchain.callbacks.manager import CallbackManager
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain_community.llms import Ollama
from langchain_core.runnables import RunnablePassthrough

RAG_TEMPLATE = """
Vous êtes un assistant qui rend le droit français plus accessible au grand public.
Utilisez les informations suivantes pour répondre à la question posée.
L'exactitude est primordiale : si vous ne connaissez pas la réponse, indiquez simplement que vous ne savez pas.
La réponse doit être en français et comporter au maximum 800 caractères.

<context>
{context}
</context>

Répondez à la question suivante:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)
llm = Ollama(model="mistral:latest", verbose=True, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))
chain = (
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    | rag_prompt
    | llm
    | StrOutputParser()
)
print("Chain loaded")

question = "Je suis serveur dans un bar, est-ce que mon patron peut me forcer à faire des heures supp ?"

docs = chroma_store.similarity_search(question)
print("Context loaded")

# Run
print("Running chain")
chain.invoke({"context": docs, "question": question})

 Bien sûr, je tente de répondre avec précision. Cependant, il n'y a pas de réponse exhaustive sans connaître les détails contratuellement et juridiquement de votre situation. En général, le travail supplémentaire (heures supplémentaires ou "heures sup") est une question complexe en matière de droit du travail français. La loi oblige l'employeur à offrir des indemnités pour ces heures supplémentaires (article L. 3121-2 du Code du Travail). Si votre contrat de travail ne mentionne rien d'heures supplémentaires et que vous n'avez pas consenti à travailler en sus de vos heures normales, votre patron ne peut vous forcer à le faire. Toutefois, si des dispositions contractuelles ou collectives existent relatives aux heures sup, il est préconisable de consulter une expertise juridique.

' Bien sûr, je tente de répondre avec précision. Cependant, il n\'y a pas de réponse exhaustive sans connaître les détails contratuellement et juridiquement de votre situation. En général, le travail supplémentaire (heures supplémentaires ou "heures sup") est une question complexe en matière de droit du travail français. La loi oblige l\'employeur à offrir des indemnités pour ces heures supplémentaires (article L. 3121-2 du Code du Travail). Si votre contrat de travail ne mentionne rien d\'heures supplémentaires et que vous n\'avez pas consenti à travailler en sus de vos heures normales, votre patron ne peut vous forcer à le faire. Toutefois, si des dispositions contractuelles ou collectives existent relatives aux heures sup, il est préconisable de consulter une expertise juridique.'